In [1]:
import time
import datetime
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

# CONSTS

In [2]:
CHROME_DRIVER_PATH = "./chromedriver-linux/chromedriver"
BRAVE_PATH = "/usr/bin/brave-browser"
DOWNLOAD_DIRECTORY = "/home/mamoudou/dossier_partage/donnees-meteo/download"
ARCHIVE_DIRECTORY = "/home/mamoudou/dossier_partage/donnees-meteo/archive/daily"

In [3]:
now = datetime.datetime.now()

formatted_date = now.strftime("%Y%m%d %H").split(" ")

DATE, HEURE = formatted_date[0], "{:02d}".format(int(formatted_date[1])-2)
print(f"Voici la date du jour recuperer: {formatted_date}.\nAprès répartiton, on a une date: {DATE} et une heure: {HEURE}")

URL = f"https://donneespubliques.meteofrance.fr/?fond=donnee_libre&prefixe=Txt%2FSynop%2Fsynop&extension=csv&date={DATE}&reseau={HEURE}"

print(f"L'URL formée après, devient: {URL}")

Voici la date du jour recuperer: ['20250202', '11'].
Après répartiton, on a une date: 20250202 et une heure: 09
L'URL formée après, devient: https://donneespubliques.meteofrance.fr/?fond=donnee_libre&prefixe=Txt%2FSynop%2Fsynop&extension=csv&date=20250202&reseau=09


## Browser Configs

In [4]:
options = Options()
options.binary_location = BRAVE_PATH  
options.add_argument("--headless=new")
options.add_argument("--disable-gpu")
options.add_argument("--nosandbox")
options.add_argument("--disable-dev-shm-usage")

prefs = {
    "download.default_directory": DOWNLOAD_DIRECTORY,
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "safebrowsing.enabled": True 
}
options.add_experimental_option("prefs", prefs)

In [5]:
service = Service(executable_path=CHROME_DRIVER_PATH)

driver = webdriver.Chrome(service=service, options=options)

try:
    print(f"Téléchargement pour la date : {DATE}")
    driver.get(URL)
    time.sleep(5)
    print(f"Téléchargement terminé !!!")

except Exception as e:
    print(f"Erreur lors du téléchargement de {DATE}: {e}")

driver.close()
driver.quit()

Téléchargement pour la date : 20250202


Téléchargement terminé !!!


## Fixing the file extension problem

In [6]:
from file_management import change_file_extension

cvs_file = change_file_extension(DOWNLOAD_DIRECTORY)
print(f"The file I'm working with: {cvs_file}")

Le fichier a été renommé : /home/mamoudou/dossier_partage/donnees-meteo/download/synop.2025020209.txt -> /home/mamoudou/dossier_partage/donnees-meteo/download/synop.2025020209.csv
The file I'm working with: /home/mamoudou/dossier_partage/donnees-meteo/download/synop.2025020209.csv


# Starting the ELT

In [7]:
import pandas as pd
import numpy as np
df = pd.read_csv(cvs_file, sep=";", dtype={"date": str})

print(df.to_string())

    numer_sta            date    pmer  tend cod_tend   dd         ff           t          td    u     vv  ww  w1  w2    n nbas  hbas  cl  cm  ch    pres niv_bar geop tend24 tn12        tn24 tx12        tx24     tminsol  sw  tw      raf10     rafper  per etat_sol  ht_neige    ssfrai perssfrai        rr1        rr3        rr6       rr12       rr24 phenspe1 phenspe2 phenspe3 phenspe4 nnuage1 ctype1 hnuage1 nnuage2 ctype2 hnuage2 nnuage3 ctype3 hnuage3 nnuage4 ctype4 hnuage4  Unnamed: 59
0        7005  20250202090000  102380    20        2  120   2.100000  273.150000  271.550000   89  13180   0  mq  mq   mq   mq    mq  mq  mq  mq  101470      mq   mq   -790   mq          mq   mq          mq  271.950000  mq  mq   4.600000   4.600000  -10       mq  0.000000        mq        mq   0.000000   0.000000   0.000000   0.000000   0.000000       mq       mq       mq       mq      mq     mq      mq      mq     mq      mq      mq     mq      mq      mq     mq      mq          NaN
1        7015  2025020

# Récupération des informations importantes à savoir :
1. Indicatif OMM station : numéro de station `numer_sta` -> `int`
2. Date (UTC) : `date` -> `datetime`
3. Pression au niveau mer : `pmer` -> `int`
4. Variation de pression en 3 heures : `tend` -> `int`
5. Type de tendance barométrique : `cod_tend` -> `int`
6. Direction du vent moyen 10 mn : `dd` -> `int`
7. Vitesse du vent moyen 10 mn : `ff` -> `float`
8. Température : `t` -> `float`
9. Point de rosée : `td` -> `float`
10. Humidité : `u` -> `int`
11. Visibilité horizontale : `vv` -> `float`
12. Temps présent : `ww` -> `int`
13. Nébulosité totale : `n` -> `float`
14. Nébulosité des nuages de l'étage inférieur : `nbas` -> `int`
15. Hauteur de la base des nuages de l'étage inférieur : `hbas` -> `int`
16. Pression station : `pres` -> `int`
17. Variation de pression en 24 heures : `tend24` -> `int`
18. Température minimale sur N heures : `tn12` -> `float`
19. Température maximale sur N heures : `tx12` -> `float`
20. Température minimale du sol sur 12 heures : `tminsol` -> `float`
21. Rafales sur les 10 dernières minutes : `raf10` -> `float`
22. Rafales sur une période : `rafper` -> `float`
23. Période de mesure des rafales : `per` -> `float`
24. Précipitations dans les N dernières heures : `rr12` -> `float`

In [8]:
df = df[['numer_sta', 'date', "pmer", "tend", "cod_tend", "dd", "ff", "t", "td", "u", "vv", "ww", "n", "nbas", "hbas","pres", "tend24", "tn12", "tx12", "tminsol", "raf10", "rafper", "per", "rr12" ]]


print(df.to_string())


    numer_sta            date    pmer  tend cod_tend   dd         ff           t          td    u     vv  ww    n nbas  hbas    pres tend24 tn12 tx12     tminsol      raf10     rafper  per       rr12
0        7005  20250202090000  102380    20        2  120   2.100000  273.150000  271.550000   89  13180   0   mq   mq    mq  101470   -790   mq   mq  271.950000   4.600000   4.600000  -10   0.000000
1        7015  20250202090000  102450     0        5  110   2.100000  272.450000  271.850000   96   5980   0   mq    0    mq  101850   -780   mq   mq  269.750000   2.600000   2.600000  -10   0.000000
2        7020  20250202090000  102040   -10        8  170   4.600000  276.850000  275.350000   90  15000   1   10    1   450  101930   -820   mq   mq          mq   7.000000   7.400000  -10   0.000000
3        7027  20250202090000  102210   -10        5  150   1.800000  273.050000  272.350000   95   3200  10   mq    0    mq  101360   -780   mq   mq  275.450000   2.500000   5.800000  -10   0.000000


## Remplacer les valeurs manquantes `mq` par `NaN`

In [9]:
df.replace("mq", np.nan, inplace=True)
print(df.to_string())

    numer_sta            date    pmer  tend cod_tend   dd         ff           t          td    u     vv   ww    n nbas  hbas    pres tend24  tn12  tx12     tminsol      raf10     rafper  per       rr12
0        7005  20250202090000  102380    20        2  120   2.100000  273.150000  271.550000   89  13180    0  NaN  NaN   NaN  101470   -790   NaN   NaN  271.950000   4.600000   4.600000  -10   0.000000
1        7015  20250202090000  102450     0        5  110   2.100000  272.450000  271.850000   96   5980    0  NaN    0   NaN  101850   -780   NaN   NaN  269.750000   2.600000   2.600000  -10   0.000000
2        7020  20250202090000  102040   -10        8  170   4.600000  276.850000  275.350000   90  15000    1   10    1   450  101930   -820   NaN   NaN         NaN   7.000000   7.400000  -10   0.000000
3        7027  20250202090000  102210   -10        5  150   1.800000  273.050000  272.350000   95   3200   10  NaN    0   NaN  101360   -780   NaN   NaN  275.450000   2.500000   5.800000  

/tmp/ipykernel_1820102/3864656540.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace("mq", np.nan, inplace=True)


## Nettoyages de colonnes non renseignées 

1. Informations générales
	-	Numéro de station : `numer_sta` -> int
	-	Date (UTC) : `date` -> datetime
2. Pression
	-	Pression au niveau mer : `pmer` -> int
	-	Variation de pression en 3 heures : `tend` -> int
	-	Type de tendance barométrique : `cod_tend` -> int
	-	Pression station : `pres` -> int
	-	Variation de pression en 24 heures : `tend24` -> int
3. Vent
	-	Direction du vent moyen 10 mn : `dd` -> int
	-	Vitesse du vent moyen 10 mn : `ff` -> float
	-	Rafales sur les 10 dernières minutes : `raf10` -> float
	-	Rafales sur une période : `rafper` -> float
	-	Période de mesure des rafales : `per` -> float
4. Température
	-	Température : `t` -> float
	-	Point de rosée : `td` -> float
	-	Température minimale sur N heures : `tn12` -> float
	-	Température maximale sur N heures : `tx12` -> float
	-	Température minimale du sol sur 12 heures : `tminsol` -> float
5. Humidité
	-	Humidité : `u` -> int
6. Visibilité
	-	Visibilité horizontale : `vv` -> float
7. Nébulosité et nuages
	-	Nébulosité totale : `n` -> float
	-	Nébulosité des nuages de l’étage inférieur : `nbas` -> int
	-	Hauteur de la base des nuages de l’étage inférieur : `hbas` -> int
8. Précipitations
	-	Précipitations dans les N dernières heures : `rr12` -> float
9. Temps présent
	-	Temps présent : `ww` -> int

In [10]:
df[['dd', 'ff', 'raf10', 'rafper', 'rr12', "per"]] = df[['dd', 'ff', 'raf10', 'rafper', 'rr12', "per"]].fillna(0)

moyenne = ["pmer", "tend", "cod_tend", "pres", "tend24", "t", "td", "tn12", "tx12", "tminsol", "u", "vv", "n", "nbas", "hbas", "ww"]
df[moyenne] = df[moyenne].apply(pd.to_numeric, errors='coerce')

for col in moyenne:
    df[col] = df[col].fillna(df[col].mean())

print(df.to_string())



    numer_sta            date       pmer        tend  cod_tend   dd         ff           t          td           u            vv         ww           n      nbas         hbas           pres       tend24  tn12  tx12     tminsol      raf10     rafper  per       rr12
0        7005  20250202090000  102380.00   20.000000  2.000000  120   2.100000  273.150000  271.550000   89.000000  13180.000000   0.000000   79.090909  3.615385  1046.086957  101470.000000  -790.000000   NaN   NaN  271.950000   4.600000   4.600000  -10   0.000000
1        7015  20250202090000  102450.00    0.000000  5.000000  110   2.100000  272.450000  271.850000   96.000000   5980.000000   0.000000   79.090909  0.000000  1046.086957  101850.000000  -780.000000   NaN   NaN  269.750000   2.600000   2.600000  -10   0.000000
2        7020  20250202090000  102040.00  -10.000000  8.000000  170   4.600000  276.850000  275.350000   90.000000  15000.000000   1.000000   10.000000  1.000000   450.000000  101930.000000  -820.000000   

## Typages des colonnes

In [11]:
df["date"] = pd.to_datetime(df["date"], format='%Y%m%d%H%M%S')
df['date'] = df['date'].dt.strftime("%Y-%m-%d %H:%M:%S")

entier = ["numer_sta", "pmer", "tend", "cod_tend", "dd", "u", "ww", "nbas", "hbas", "pres", "tend24"]
for i in entier:
    print(f"{i} -> {df[i].dtype}")
    df[i] = df[i].astype(int)
    print(f"{i} -> {df[i].dtype}")
    print("")

double = ['dd', 'ff', 'raf10', 'rafper', 'rr12', "per", "t", "td", "tn12", "tx12", "tminsol", "n", "vv"]
for i in double:
    print(f"{i} -> {df[i].dtype}")
    df[i] = df[i].astype(float)
    print(f"{i} -> {df[i].dtype}")
    print("")

numer_sta -> int64
numer_sta -> int64

pmer -> float64
pmer -> int64

tend -> float64
tend -> int64

cod_tend -> float64
cod_tend -> int64

dd -> object
dd -> int64

u -> float64
u -> int64

ww -> float64
ww -> int64

nbas -> float64
nbas -> int64

hbas -> float64
hbas -> int64

pres -> float64
pres -> int64

tend24 -> float64
tend24 -> int64

dd -> int64
dd -> float64

ff -> object
ff -> float64

raf10 -> object
raf10 -> float64

rafper -> object
rafper -> float64

rr12 -> object
rr12 -> float64

per -> object
per -> float64

t -> float64
t -> float64

td -> float64
td -> float64

tn12 -> float64
tn12 -> float64

tx12 -> float64
tx12 -> float64

tminsol -> float64
tminsol -> float64

n -> float64
n -> float64

vv -> float64
vv -> float64



In [12]:
print(df.to_string())

    numer_sta                 date    pmer  tend  cod_tend     dd    ff           t          td    u            vv  ww           n  nbas  hbas    pres  tend24  tn12  tx12     tminsol  raf10  rafper   per  rr12
0        7005  2025-02-02 09:00:00  102380    20         2  120.0   2.1  273.150000  271.550000   89  13180.000000   0   79.090909     3  1046  101470    -790   NaN   NaN  271.950000    4.6     4.6 -10.0   0.0
1        7015  2025-02-02 09:00:00  102450     0         5  110.0   2.1  272.450000  271.850000   96   5980.000000   0   79.090909     0  1046  101850    -780   NaN   NaN  269.750000    2.6     2.6 -10.0   0.0
2        7020  2025-02-02 09:00:00  102040   -10         8  170.0   4.6  276.850000  275.350000   90  15000.000000   1   10.000000     1   450  101930    -820   NaN   NaN  274.884286    7.0     7.4 -10.0   0.0
3        7027  2025-02-02 09:00:00  102210   -10         5  150.0   1.8  273.050000  272.350000   95   3200.000000  10   79.090909     0  1046  101360    -780  

## Conversion des températures de Kelvin en Celcuis

In [13]:
celcuis = ["t", "td", "tn12", "tx12", "tminsol"]
for i in celcuis:
    df[i] = df[i] - 273.15

print(df.to_string())

    numer_sta                 date    pmer  tend  cod_tend     dd    ff          t         td    u            vv  ww           n  nbas  hbas    pres  tend24  tn12  tx12    tminsol  raf10  rafper   per  rr12
0        7005  2025-02-02 09:00:00  102380    20         2  120.0   2.1   0.000000  -1.600000   89  13180.000000   0   79.090909     3  1046  101470    -790   NaN   NaN  -1.200000    4.6     4.6 -10.0   0.0
1        7015  2025-02-02 09:00:00  102450     0         5  110.0   2.1  -0.700000  -1.300000   96   5980.000000   0   79.090909     0  1046  101850    -780   NaN   NaN  -3.400000    2.6     2.6 -10.0   0.0
2        7020  2025-02-02 09:00:00  102040   -10         8  170.0   4.6   3.700000   2.200000   90  15000.000000   1   10.000000     1   450  101930    -820   NaN   NaN   1.734286    7.0     7.4 -10.0   0.0
3        7027  2025-02-02 09:00:00  102210   -10         5  150.0   1.8  -0.100000  -0.800000   95   3200.000000  10   79.090909     0  1046  101360    -780   NaN   NaN   2

## Arrondir tous les decimaux à 2chiffres après la virgules

In [14]:
df[double] = df[double].round(2)
print(df.to_string())

    numer_sta                 date    pmer  tend  cod_tend     dd    ff      t     td    u        vv  ww       n  nbas  hbas    pres  tend24  tn12  tx12  tminsol  raf10  rafper   per  rr12
0        7005  2025-02-02 09:00:00  102380    20         2  120.0   2.1   0.00  -1.60   89  13180.00   0   79.09     3  1046  101470    -790   NaN   NaN    -1.20    4.6     4.6 -10.0   0.0
1        7015  2025-02-02 09:00:00  102450     0         5  110.0   2.1  -0.70  -1.30   96   5980.00   0   79.09     0  1046  101850    -780   NaN   NaN    -3.40    2.6     2.6 -10.0   0.0
2        7020  2025-02-02 09:00:00  102040   -10         8  170.0   4.6   3.70   2.20   90  15000.00   1   10.00     1   450  101930    -820   NaN   NaN     1.73    7.0     7.4 -10.0   0.0
3        7027  2025-02-02 09:00:00  102210   -10         5  150.0   1.8  -0.10  -0.80   95   3200.00  10   79.09     0  1046  101360    -780   NaN   NaN     2.30    2.5     5.8 -10.0   0.0
4        7037  2025-02-02 09:00:00  102330    20       

# Add verif to check is all values

In [15]:
verif = ["pmer", "tend", "cod_tend", "pres", "tend24", "t", "td", "tn12", "tx12", "tminsol", "u", "vv", "n", "nbas", "hbas", "ww"]

for col in verif:
    if df[col].isna().all() :
        df[col].fillna(0.0,inplace=True)

print(df.to_string())

    numer_sta                 date    pmer  tend  cod_tend     dd    ff      t     td    u        vv  ww       n  nbas  hbas    pres  tend24  tn12  tx12  tminsol  raf10  rafper   per  rr12
0        7005  2025-02-02 09:00:00  102380    20         2  120.0   2.1   0.00  -1.60   89  13180.00   0   79.09     3  1046  101470    -790   0.0   0.0    -1.20    4.6     4.6 -10.0   0.0
1        7015  2025-02-02 09:00:00  102450     0         5  110.0   2.1  -0.70  -1.30   96   5980.00   0   79.09     0  1046  101850    -780   0.0   0.0    -3.40    2.6     2.6 -10.0   0.0
2        7020  2025-02-02 09:00:00  102040   -10         8  170.0   4.6   3.70   2.20   90  15000.00   1   10.00     1   450  101930    -820   0.0   0.0     1.73    7.0     7.4 -10.0   0.0
3        7027  2025-02-02 09:00:00  102210   -10         5  150.0   1.8  -0.10  -0.80   95   3200.00  10   79.09     0  1046  101360    -780   0.0   0.0     2.30    2.5     5.8 -10.0   0.0
4        7037  2025-02-02 09:00:00  102330    20       

/tmp/ipykernel_1820102/555783230.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(0.0,inplace=True)


# Archive file

In [16]:
from file_management import move_file

move_file(cvs_file, ARCHIVE_DIRECTORY)

Le fichier a été déplacé de /home/mamoudou/dossier_partage/donnees-meteo/download/synop.2025020209.csv à /home/mamoudou/dossier_partage/donnees-meteo/archive/daily/synop.2025020209.csv


# Sending data to API

In [17]:
import requests
from api import send_data

In [18]:
API_URL = "http://127.0.0.1:8080/api/donnees-climatiques" 

dict_data = df.to_dict(orient="records")

for row in dict_data:
    print(row)
    send_data(API_URL, row, requests)

{'numer_sta': 7005, 'date': '2025-02-02 09:00:00', 'pmer': 102380, 'tend': 20, 'cod_tend': 2, 'dd': 120.0, 'ff': 2.1, 't': 0.0, 'td': -1.6, 'u': 89, 'vv': 13180.0, 'ww': 0, 'n': 79.09, 'nbas': 3, 'hbas': 1046, 'pres': 101470, 'tend24': -790, 'tn12': 0.0, 'tx12': 0.0, 'tminsol': -1.2, 'raf10': 4.6, 'rafper': 4.6, 'per': -10.0, 'rr12': 0.0}


Données ajoutées avec succès!
{'numer_sta': 7015, 'date': '2025-02-02 09:00:00', 'pmer': 102450, 'tend': 0, 'cod_tend': 5, 'dd': 110.0, 'ff': 2.1, 't': -0.7, 'td': -1.3, 'u': 96, 'vv': 5980.0, 'ww': 0, 'n': 79.09, 'nbas': 0, 'hbas': 1046, 'pres': 101850, 'tend24': -780, 'tn12': 0.0, 'tx12': 0.0, 'tminsol': -3.4, 'raf10': 2.6, 'rafper': 2.6, 'per': -10.0, 'rr12': 0.0}
Données ajoutées avec succès!
{'numer_sta': 7020, 'date': '2025-02-02 09:00:00', 'pmer': 102040, 'tend': -10, 'cod_tend': 8, 'dd': 170.0, 'ff': 4.6, 't': 3.7, 'td': 2.2, 'u': 90, 'vv': 15000.0, 'ww': 1, 'n': 10.0, 'nbas': 1, 'hbas': 450, 'pres': 101930, 'tend24': -820, 'tn12': 0.0, 'tx12': 0.0, 'tminsol': 1.73, 'raf10': 7.0, 'rafper': 7.4, 'per': -10.0, 'rr12': 0.0}
Données ajoutées avec succès!
{'numer_sta': 7027, 'date': '2025-02-02 09:00:00', 'pmer': 102210, 'tend': -10, 'cod_tend': 5, 'dd': 150.0, 'ff': 1.8, 't': -0.1, 'td': -0.8, 'u': 95, 'vv': 3200.0, 'ww': 10, 'n': 79.09, 'nbas': 0, 'hbas': 1046, 'pres': 101360, 'te

{'numer_sta': 7037, 'date': '2025-02-02 09:00:00', 'pmer': 102330, 'tend': 20, 'cod_tend': 0, 'dd': 120.0, 'ff': 2.5, 't': -0.2, 'td': -1.3, 'u': 92, 'vv': 6720.0, 'ww': 0, 'n': 79.09, 'nbas': 0, 'hbas': 1046, 'pres': 100350, 'tend24': -800, 'tn12': 0.0, 'tx12': 0.0, 'tminsol': 0.6, 'raf10': 3.7, 'rafper': 4.1, 'per': -10.0, 'rr12': 0.0}
Données ajoutées avec succès!
{'numer_sta': 7072, 'date': '2025-02-02 09:00:00', 'pmer': 102460, 'tend': 20, 'cod_tend': 2, 'dd': 110.0, 'ff': 1.5, 't': -0.7, 'td': -2.1, 'u': 90, 'vv': 13450.0, 'ww': 0, 'n': 79.09, 'nbas': 0, 'hbas': 1046, 'pres': 101250, 'tend24': -670, 'tn12': 0.0, 'tx12': 0.0, 'tminsol': -0.6, 'raf10': 2.3, 'rafper': 2.6, 'per': -10.0, 'rr12': 0.0}
Données ajoutées avec succès!
{'numer_sta': 7110, 'date': '2025-02-02 09:00:00', 'pmer': 101730, 'tend': 30, 'cod_tend': 3, 'dd': 150.0, 'ff': 7.2, 't': 6.1, 'td': 4.9, 'u': 92, 'vv': 19590.0, 'ww': 0, 'n': 100.0, 'nbas': 8, 'hbas': 250, 'pres': 100520, 'tend24': -1130, 'tn12': 0.0, 'tx1

Données ajoutées avec succès!
{'numer_sta': 7181, 'date': '2025-02-02 09:00:00', 'pmer': 102490, 'tend': 50, 'cod_tend': 0, 'dd': 220.0, 'ff': 0.7, 't': 0.4, 'td': -0.9, 'u': 91, 'vv': 10330.0, 'ww': 0, 'n': 79.09, 'nbas': 0, 'hbas': 1046, 'pres': 98140, 'tend24': -490, 'tn12': 0.0, 'tx12': 0.0, 'tminsol': -5.5, 'raf10': 1.5, 'rafper': 1.7, 'per': -10.0, 'rr12': 0.0}
Données ajoutées avec succès!
{'numer_sta': 7190, 'date': '2025-02-02 09:00:00', 'pmer': 102570, 'tend': 80, 'cod_tend': 0, 'dd': 20.0, 'ff': 1.7, 't': -0.2, 'td': -1.6, 'u': 90, 'vv': 8880.0, 'ww': 0, 'n': 100.0, 'nbas': 8, 'hbas': 800, 'pres': 100660, 'tend24': -480, 'tn12': 0.0, 'tx12': 0.0, 'tminsol': -3.6, 'raf10': 2.7, 'rafper': 3.6, 'per': -10.0, 'rr12': 0.0}
Données ajoutées avec succès!
{'numer_sta': 7207, 'date': '2025-02-02 09:00:00', 'pmer': 101673, 'tend': 12, 'cod_tend': 4, 'dd': 140.0, 'ff': 9.9, 't': 7.8, 'td': 5.98, 'u': 87, 'vv': 16128.22, 'ww': 6, 'n': 79.09, 'nbas': 3, 'hbas': 1046, 'pres': 100037, 'ten

{'numer_sta': 7314, 'date': '2025-02-02 09:00:00', 'pmer': 102110, 'tend': -40, 'cod_tend': 6, 'dd': 140.0, 'ff': 2.0, 't': 3.1, 'td': 2.1, 'u': 93, 'vv': 10000.0, 'ww': 1, 'n': 10.0, 'nbas': 1, 'hbas': 2250, 'pres': 101820, 'tend24': -610, 'tn12': 0.0, 'tx12': 0.0, 'tminsol': 1.73, 'raf10': 3.1, 'rafper': 3.2, 'per': -10.0, 'rr12': 0.0}
Données ajoutées avec succès!
{'numer_sta': 7335, 'date': '2025-02-02 09:00:00', 'pmer': 102190, 'tend': -20, 'cod_tend': 8, 'dd': 160.0, 'ff': 0.9, 't': -0.8, 'td': -0.9, 'u': 99, 'vv': 190.0, 'ww': 49, 'n': 101.0, 'nbas': 9, 'hbas': 30, 'pres': 100670, 'tend24': -650, 'tn12': 0.0, 'tx12': 0.0, 'tminsol': 3.1, 'raf10': 1.7, 'rafper': 2.2, 'per': -10.0, 'rr12': 0.0}
Données ajoutées avec succès!
{'numer_sta': 7434, 'date': '2025-02-02 09:00:00', 'pmer': 102220, 'tend': 20, 'cod_tend': 2, 'dd': 30.0, 'ff': 2.2, 't': 1.4, 'td': -0.1, 'u': 90, 'vv': 14400.0, 'ww': 0, 'n': 79.09, 'nbas': 0, 'hbas': 1046, 'pres': 97260, 'tend24': -480, 'tn12': 0.0, 'tx12': 

{'numer_sta': 7591, 'date': '2025-02-02 09:00:00', 'pmer': 101673, 'tend': 50, 'cod_tend': 1, 'dd': 70.0, 'ff': 3.7, 't': 7.6, 'td': -1.1, 'u': 54, 'vv': 16128.22, 'ww': 6, 'n': 79.09, 'nbas': 3, 'hbas': 1046, 'pres': 91730, 'tend24': -250, 'tn12': 0.0, 'tx12': 0.0, 'tminsol': 1.73, 'raf10': 8.8, 'rafper': 8.8, 'per': -10.0, 'rr12': 0.0}
Données ajoutées avec succès!
{'numer_sta': 7607, 'date': '2025-02-02 09:00:00', 'pmer': 102130, 'tend': -60, 'cod_tend': 8, 'dd': 90.0, 'ff': 1.6, 't': 1.3, 'td': 1.3, 'u': 100, 'vv': 19410.0, 'ww': 0, 'n': 79.09, 'nbas': 0, 'hbas': 1046, 'pres': 101390, 'tend24': -370, 'tn12': 0.0, 'tx12': 0.0, 'tminsol': 2.5, 'raf10': 2.6, 'rafper': 3.1, 'per': -10.0, 'rr12': 0.0}
Données ajoutées avec succès!
{'numer_sta': 7621, 'date': '2025-02-02 09:00:00', 'pmer': 102130, 'tend': -40, 'cod_tend': 8, 'dd': 130.0, 'ff': 1.0, 't': 4.6, 'td': 2.5, 'u': 86, 'vv': 32860.0, 'ww': 0, 'n': 79.09, 'nbas': 0, 'hbas': 1046, 'pres': 97690, 'tend24': -320, 'tn12': 0.0, 'tx12'

{'numer_sta': 61970, 'date': '2025-02-02 09:00:00', 'pmer': 100810, 'tend': 80, 'cod_tend': 8, 'dd': 260.0, 'ff': 2.1, 't': 33.8, 'td': 25.8, 'u': 63, 'vv': 16128.22, 'ww': 6, 'n': 79.09, 'nbas': 3, 'hbas': 1046, 'pres': 100700, 'tend24': -50, 'tn12': 0.0, 'tx12': 0.0, 'tminsol': 1.73, 'raf10': 0.0, 'rafper': 3.8, 'per': -10.0, 'rr12': 0.0}
Données ajoutées avec succès!
{'numer_sta': 61972, 'date': '2025-02-02 09:00:00', 'pmer': 100940, 'tend': 70, 'cod_tend': 7, 'dd': 150.0, 'ff': 5.8, 't': 31.1, 'td': 23.7, 'u': 65, 'vv': 16128.22, 'ww': 6, 'n': 79.09, 'nbas': 3, 'hbas': 1046, 'pres': 100870, 'tend24': 70, 'tn12': 0.0, 'tx12': 0.0, 'tminsol': 1.73, 'raf10': 0.0, 'rafper': 10.5, 'per': -10.0, 'rr12': 0.0}
Données ajoutées avec succès!


{'numer_sta': 61976, 'date': '2025-02-02 09:00:00', 'pmer': 100560, 'tend': 30, 'cod_tend': 8, 'dd': 0.0, 'ff': 0.0, 't': 28.3, 'td': 25.1, 'u': 83, 'vv': 16128.22, 'ww': 6, 'n': 79.09, 'nbas': 3, 'hbas': 1046, 'pres': 100470, 'tend24': -130, 'tn12': 0.0, 'tx12': 0.0, 'tminsol': 1.73, 'raf10': 0.0, 'rafper': 0.0, 'per': -10.0, 'rr12': 10.6}
Données ajoutées avec succès!
{'numer_sta': 61980, 'date': '2025-02-02 09:00:00', 'pmer': 100320, 'tend': -210, 'cod_tend': 6, 'dd': 110.0, 'ff': 12.5, 't': 33.5, 'td': 24.4, 'u': 59, 'vv': 19190.0, 'ww': 0, 'n': 79.09, 'nbas': 0, 'hbas': 1046, 'pres': 100180, 'tend24': -340, 'tn12': 0.0, 'tx12': 0.0, 'tminsol': 1.73, 'raf10': 17.7, 'rafper': 20.0, 'per': -10.0, 'rr12': 0.4}
Données ajoutées avec succès!
{'numer_sta': 61996, 'date': '2025-02-02 09:00:00', 'pmer': 101330, 'tend': 180, 'cod_tend': 8, 'dd': 330.0, 'ff': 6.9, 't': 22.8, 'td': 17.0, 'u': 70, 'vv': 16128.22, 'ww': 6, 'n': 79.09, 'nbas': 3, 'hbas': 1046, 'pres': 100990, 'tend24': -630, 'tn